# Exercise 00-2 : Prepare Data

ここでは、Azure Machine Learning のデータ アセットとしてデータをアップロードします。
この後のすべての演習（演習02～）では、この演習で用意したデータを使用しますので、事前にこの演習を実行してください。

ここでは、scikit-learn の diabetes のデータセット ([load_diabetes](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html)) を使って学習します。


## 糖尿病データセットの取得

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn import datasets
 
# Load diabetes datasets
diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target
 
# Create a pandas DataFrame that serves as sample input for the deployed ElasticNet model
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6', 'progression']
data = pd.DataFrame(d, columns=cols)
data350 = data[:350]

In [ ]:
print(data.shape)
print(data350.shape)
data

In [ ]:
os.makedirs("data", exist_ok=True)

data.to_csv("data/diabetes_data.csv", index=False)
data350.to_csv("data/diabetes_data_350.csv", index=False)

## デフォルトのデータストアにローカルファイルをアップロードする
Azure Machine Learning（AML）ワークスペースは、独自のデフォルトデータストアを持ちます。AML ワークスペースを作成すると、同じリソースグループ内にストレージアカウント（デフォルトデータストア）が自動生成されます。
この例では、yaml を作成し、ファイル（data フォルダ内）を AML にアップロードします。


まず、データ アセット登録のための yaml を作成します。

In [ ]:
%%writefile 00_file_upload.yml
$schema: https://azuremlschemas.azureedge.net/latest/data.schema.json
name: diabetes_data_oh4ml
description: diabades data for oh4ml
type: uri_file
path: data/diabetes_data.csv

ここで、AML CLI v2 でデータ アセットを登録（ローカルデータのアップロード）します。

In [ ]:
!az ml data create --file 00_file_upload.yml

## カラム数を減らしたデータ アセットの準備
データサイエンス初期のデータが少ない状態をシミュレートするため、あえてレコードを減らしたバージョンのデータ アセットを作成します。

In [ ]:
%%writefile 00_file_upload_350.yml
$schema: https://azuremlschemas.azureedge.net/latest/data.schema.json
name: diabetes_data_oh4ml_350records
description: diabades data for oh4ml(350)
type: uri_file
path: data/diabetes_data_350.csv

In [ ]:
!az ml data create --file 00_file_upload_350.yml

AML Studio の左メニュー「データ」をクリックすると実際にデータ アセットが登録されていることが確認できます。